In [2]:
import numpy as np
import pandas as pd
import Levenshtein

In [62]:
def leventstein_similarity(str1:str, str2:str):
    return 1 - Levenshtein.distance(str1, str2) / max(len(str1), len(str2))

In [86]:
def same_similarity(str1:str, str2:str):
    return str1==str2

In [66]:
def tokenize_date(date):
    # Tokenize the date string into year, month, and day
    return set(date.split('-'))

def date_similarity(date1, date2, threshold=1):
    
    if date1 is None or date2 is None:
        return 0
    
    tokens1 = tokenize_date(date1)
    tokens2 = tokenize_date(date2)
    
    intersection = len(tokens1.intersection(tokens2))
    union = len(tokens1.union(tokens2))
    jaccard_similarity = intersection / union
    
    if jaccard_similarity >= threshold:
        return True
    else:
        return False

date1 = "2000-09-14"
date2 = "2000-14-09"
threshold = 0.7  # You can adjust this threshold as needed

if date_similarity(date1, date2, threshold):
    print("Dates are similar.")
else:
    print("Dates are not similar.")

Dates are similar.


In [68]:
def get_similarity_matrix(similarity_function, col1, col2):

    similarity_matrix = np.zeros((len(col1), len(col2)))

    for i, entry1 in enumerate(col1):
        for j, entry2 in enumerate(col2):

            similarity_matrix[i, j] = similarity_function(entry1, entry2) 
        # print(f"Row {i}")     

    return similarity_matrix

In [75]:
def create_result(df1, df2, similarity_matrix):
    matches = pd.DataFrame()
    matches['df1'] = df1['id']
    matches['df2'] = df2.iloc[similarity_matrix.argmax(axis=1)]['id'].values
    matches['name1'] = df1['name'].values
    matches['birthdate1'] = df1['birthdate'].values
    matches['name2'] = df2.iloc[similarity_matrix.argmax(axis=1)]['name'].values
    matches['birthdate2'] = df2.iloc[similarity_matrix.argmax(axis=1)]['birthdate'].values
    matches['score'] = similarity_matrix.max(axis=1)
    
    return matches

In [38]:
df_eafc = pd.read_xml('../Football-Project/data/xml-files/eafc_final.xml')
df_fm23 = pd.read_xml('../Football-Project/data/xml-files/fm23_final.xml')
df_tm = pd.read_xml('../Football-Project/data/xml-files/tm_final.xml')

In [39]:
df_eafc_sampled = df_eafc.sample(n=100, random_state=42)
df_fm23_sampled = df_fm23.sample(n=100, random_state=42)
df_tm_sampled = df_tm.sample(n=100, random_state=42)

In [69]:
similarity_matrix_name = get_similarity_matrix(leventstein_similarity, df_fm23_sampled['name'], df_tm['name'])
similarity_matrix_date = get_similarity_matrix(date_similarity, df_fm23_sampled['birthdate'], df_tm['birthdate'])


In [87]:
df = create_result(df_fm23_sampled, df_tm, similarity_matrix_name)

In [89]:
df[df['score']>.8]

,df1,df2,name1,birthdate1,name2,birthdate2,score
5168,fm_6168,tm_6113,Adam Marušić,1992-10-17,Adam Marusic,1992-10-17,0.833333
10897,fm_11897,tm_9870,Jamal Lowe,1994-07-21,Jamal Lowe,1994-07-21,1.000000
9034,fm_10034,tm_5313,Matheus Pereira,1996-05-05,Matheus Pereira,2000-12-21,1.000000
13264,fm_14264,tm_3397,Alex Telles,1992-12-15,Alex Telles,1992-12-15,1.000000
10546,fm_11546,tm_6064,Rasmus Kristensen,1997-07-11,Rasmus Kristensen,1997-07-11,1.000000
6072,fm_7072,tm_8731,Simone Canestrelli,2000-09-11,Simone Canestrelli,2000-09-11,1.000000
101,fm_1101,tm_2158,Alejandro Bedoya,1987-04-29,Alejandro Bedoya,1987-04-29,1.000000
1570,fm_2570,tm_5856,Aron Dønnum,1998-04-20,Aron Dønnum,1998-04-20,1.000000
13223,fm_14223,tm_7201,Pau Torres,1997-01-16,Pau Torres,1997-01-16,1.000000
9452,fm_10452,tm_2623,Mohammed Al-Saiari,1993-05-02,Mohammed Al-Khaibari,2002-04-18,0.850000


In [ ]:
print(df_eafc['name'][0], df_fm23['name'][14173])

Rúben Fernandes Rúben Fernandes
